# Create filenames, filepaths, and JSON


1. [Generating filenames](#Generating-filenames)
1. [Modifying filenames](#Modifying-filenames)
1. [Generating file names for derivatives](#Generating-file-names-for-derivatives)


In [6]:
% add bids-matlab to path
addpath(fullfile(pwd, '..'));

 ## Generating filenames

Uses the `bids.File` class.

In [16]:
name_spec.ext = '.nii';
name_spec.suffix = 'bold';
name_spec.entities = struct( ...
                          'sub', '01', ...
                          'task', 'face recognition', ...
                          'run', '02', ...
                          'ses', 'test');
file = bids.File(name_spec);

file.filename

ans = sub-01_task-faceRecognition_run-02_ses-test_bold.nii


You can rely on the BIDS schema to know in which order the entities must go for a certain `suffix` type. 

In [17]:
use_schema = true;

file = bids.File(name_spec, use_schema);

file.filename

ans = sub-01_ses-test_task-faceRecognition_run-02_bold.nii


This can also tell you if you are missing a required entity if you set `tolerant` to `false`.

In [29]:
clear

name_spec.suffix = 'bold';
name_spec.ext = '.nii';
name_spec.entities = struct( ...
                  'sub', '01', ...
                  'ses', 'test', ...
                  'run', '02');
                  
use_schema = true;

tolerant = false;

file = bids.File(name_spec, use_schema, struct(), tolerant);


error: Entities 'task' cannot not be empty for the suffix 'bold'
error: called from
    error_handling at line 44 column 5
    bidsFile_error at line 516 column 7
    check_required_entities at line 485 column 9
    concatenate_entities at line 451 column 7
    create_filename at line 285 column 14
    File at line 156 column 11


Or you can specify the order of the entities manually.

In [28]:
clear

name_spec.ext = '.nii';
name_spec.suffix = 'bold';
name_spec.entities = struct( ...
                          'sub', '01', ...
                          'task', 'face recognition', ...
                          'run', '02', ...
                          'ses', 'test');
file = bids.File(name_spec);

entity_order = {'run', 'sub', 'ses'};

file = file.reorder_entities(entity_order);
file = file.create_filename;

file.filename

ans = run-02_sub-01_ses-test_task-faceRecognition_bold.nii


## Modifying filenames

This can be used:
- to add, remove, modify any of the entities
- change the suffix
- change the extensions
- add or remove any prefix


In [34]:
clear

filename = 'sub-01_ses-mri_T1w.nii';

use_schema = false;

name_spec.suffix = 'mask';
name_spec.entities = struct('ses', '', ...
                            'desc', 'brain');

file = bids.File(filename, use_schema, name_spec);

file.filename

ans = sub-01_desc-brain_mask.nii


## Generating file names for derivatives

This can also be useful to remove the prefix of some files.

In [37]:
clear

filename = 'wuasub-01_ses-test_task-faceRecognition_run-02_bold.nii';

use_schema = false;

name_spec.prefix = '';
name_spec.entities = struct('space', 'IXI549Space', ...
                            'desc', 'preproc');

file = bids.File(filename, use_schema, name_spec);

file.filename

ans = sub-01_ses-test_task-faceRecognition_run-02_space-IXI549Space_desc-preproc_bold.nii


This can prove useful to get a dummy json that should accompany any derivatives files.

In [42]:
json = bids.derivatives_json(file.filename);
json.filename
json.content

ans = sub-01_ses-test_task-faceRecognition_run-02_space-IXI549Space_desc-preproc_bold.json
ans =

  scalar structure containing the fields:

    Description = RECOMMENDED
    Sources = 
    {
      [1,1] = 
      {
        [1,1] = OPTIONAL
      }
    }
    RawSources = 
    {
      [1,1] = 
      {
        [1,1] = OPTIONAL
      }
    }
    SpatialReference = 
    {
      [1,1] = 
      {
        [1,1] = REQUIRED if no space entityor if non standard space RECOMMENDED otherwise
      }
    }



The content of the JSON should adapt depending on the entities or suffix present in the output filename.

In [46]:
json = bids.derivatives_json('sub-01_ses-test_task-faceRecognition_res-2pt0_space-IXI549Space_desc-brain_mask.nii')
json.filename
json.content
json.content.Resolution{1}{1}

json =

  scalar structure containing the fields:

    filename = sub-01_ses-test_task-faceRecognition_res-2pt0_space-IXI549Space_desc-brain_mask.json
    content =

      scalar structure containing the fields:

        Description = RECOMMENDED
        Sources = 
        {
          [1,1] = 
          {
            [1,1] = OPTIONAL
          }
        }
        RawSources = 
        {
          [1,1] = 
          {
            [1,1] = REQUIRED
          }
        }
        SpatialReference = 
        {
          [1,1] = 
          {
            [1,1] = REQUIRED if no space entity or if non standard space RECOMMENDED otherwise
          }
        }
        Resolution = 
        {
          [1,1] = 
          {
            [1,1] =

              scalar structure containing the fields:

                2pt0: 1x24 sq_string

          }
        }
        Atlas = 
        {
          [1,1] = 
          {
            [1,1] = OPTIONAL
          }
        }
        Type = 
        {
        